In [1]:
import tensorflow as tf

In [2]:
tf.__version__

'2.0.0-rc1'

### 加载数据 (fashion-minit)

In [3]:
def load_mnist(path, kind='train'):
    import os
    import gzip
    import numpy as np

    """Load MNIST data from `path`"""
    labels_path = os.path.join(path,
                               '%s-labels-idx1-ubyte.gz'
                               % kind)
    images_path = os.path.join(path,
                               '%s-images-idx3-ubyte.gz'
                               % kind)

    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8,
                               offset=8)

    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8,
                               offset=16).reshape(len(labels), 784)

    return images, labels

In [10]:
path = "../../data/fashion_mnist"

In [15]:
text_labels = ['t-shirt', 'trouser', 'pullover', 'dress', 'coat',
'sandal', 'shirt', 'sneaker', 'bag', 'ankle boot']

In [11]:
x_train,y_train = load_mnist(path)
x_test,y_test = load_mnist(path,"t10k")

In [16]:
x_train[0],text_labels[y_train[0]]

(array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   1,   0,   0,  13,  73,   0,   0,   1,
          4,   0,   0,   0,   0,   1,   1,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   3,   0,  36, 136, 127,  62,
         54,   0,   0,   0,   1,   3,   4,   0,   0,   3,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   6,   0, 102, 204,
        176, 134, 144, 123,  23,   0,   0,   0,   0,  12,  10,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0

### 生成dataset

In [62]:
ds_train = tf.data.Dataset.from_tensor_slices((x_train,y_train))
ds_test = tf.data.Dataset.from_tensor_slices((x_test,y_test))

In [63]:
def map_fun(x,y):
    x = tf.cast(x,tf.float32)
    return x / 255.0 ,tf.cast(y,tf.int32)

In [64]:
def transformDs(ds):
    return ds.shuffle(5000).batch(32).map(map_fun).repeat()

In [65]:
ds_train = transformDs(ds_train)
ds_test = transformDs(ds_test)

In [66]:
next(iter(ds_train))

(<tf.Tensor: id=7416, shape=(32, 784), dtype=float32, numpy=
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>,
 <tf.Tensor: id=7417, shape=(32,), dtype=int32, numpy=
 array([1, 0, 3, 4, 6, 9, 4, 6, 8, 9, 3, 0, 4, 8, 3, 2, 8, 3, 6, 4, 5, 7,
        1, 8, 4, 6, 7, 3, 7, 5, 9, 6])>)

### 创建模型

In [82]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(32,activation="relu"))#,kernel_initializer = tf.keras.initializers.he_normal()
model.add(tf.keras.layers.Dense(10,activation="softmax"))#,kernel_initializer = tf.keras.initializers.he_normal()

In [83]:
model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy,optimizer=tf.keras.optimizers.SGD(0.03),metrics=["acc"])

In [ ]:
steps_per_epoch = len(x_train) // 32
validation_steps = len(x_test) // 32
model.fit(ds_train,epochs=10,steps_per_epoch=steps_per_epoch,validation_data = ds_test,validation_steps=validation_steps)

Train for 1875 steps, validate for 312 steps
Epoch 1/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.6206 - acc: 0.7857 - val_loss: 0.5351 - val_acc: 0.8147
Epoch 2/10
1875/1875 [==============================] - 5s 2ms/step - loss: 0.4662 - acc: 0.8362 - val_loss: 0.4678 - val_acc: 0.8332
Epoch 3/10
1875/1875 [==============================] - 5s 2ms/step - loss: 0.4283 - acc: 0.8486 - val_loss: 0.4433 - val_acc: 0.8414
Epoch 4/10
1875/1875 [==============================] - 5s 2ms/step - loss: 0.4054 - acc: 0.8580 - val_loss: 0.4226 - val_acc: 0.8512
Epoch 5/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3881 - acc: 0.8634 - val_loss: 0.4080 - val_acc: 0.8553
Epoch 6/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3749 - acc: 0.8675 - val_loss: 0.4021 - val_acc: 0.8582
Epoch 7/10
  73/1875 [>.............................] - ETA: 4s - loss: 0.3627 - acc: 0.8682